In [1]:
pip install demucs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.1/87.1 kB 8.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 6.7 MB/s eta 0:00:00
  Created wheel for demucs: filename=demucs-4.0.1-py3-none-any.whl size=78388 sha256=2f66c1c894ef8f1c7cbed02e04d80a8fe1109d6a72f1b10c3654d9a5856b5849
  Stored in directory: /root/.cache/pip/wheels/1b/0c/20/a3b3daa1f9b65c8b0445729f94740ec335d0f86f1066c5c414
  Created wheel for julius: filename=julius-0.2.7-

In [2]:
import os
import zipfile
from demucs.separate import main

# ---- CONFIG ----
zip_path = "/content/cn songs.zip"   # your zip with 20 songs
extract_dir = "/content/songs_dataset"    # where to extract songs
output_dir = "/content/output"            # where to save separated files

# 1. Unzip all songs
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"✅ Extracted all songs to {extract_dir}")

# 2. Get all audio files from extracted folder
audio_files = []
for root, dirs, files in os.walk(extract_dir):
    for file in files:
        if file.endswith(('.mp3', '.wav', '.flac', '.ogg')):  # supported formats
            audio_files.append(os.path.join(root, file))

print(f"🎶 Found {len(audio_files)} songs in dataset.")

# 3. Run Demucs on each audio file
for idx, input_audio in enumerate(audio_files, 1):
    print(f"🔄 Processing {idx}/{len(audio_files)}: {os.path.basename(input_audio)}")

    main([
        "-n", "htdemucs_ft",        # Faster model
        "--two-stems", "vocals",    # Only vocals vs music
        "--mp3",                    # Save as MP3
        "--jobs", "4",              # Parallel jobs
        "--device", "cuda",         # Use GPU (change to "cpu" if no GPU)
        "-o", output_dir,
        input_audio
    ])

print(f"\n✅ All {len(audio_files)} songs processed! Separated files saved in {output_dir}")


FileNotFoundError: [Errno 2] No such file or directory: '/content/cn songs.zip'

In [ ]:
import os
import librosa
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

# Paths
separated_dir = "/content/output/htdemucs_ft"
csv_output = "/content/dataset.csv"

# Function: Extract MFCC features
def extract_features(audio_path, n_mfcc=13):
    try:
        y, sr = librosa.load(audio_path, sr=None, mono=True)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        mfccs_mean = np.mean(mfccs, axis=1)  # average MFCCs
        return mfccs_mean
    except Exception as e:
        print(f"❌ Error with {audio_path}: {e}")
        return np.zeros(n_mfcc)

# Collect dataset
data = []
# Iterate through the song directories within the separated_dir
for song_name in os.listdir(separated_dir):
    song_path = os.path.join(separated_dir, song_name)
    if not os.path.isdir(song_path):
        continue

    # Assuming the separated files are directly inside the song folder
    vocals_file = os.path.join(song_path, "vocals.mp3")
    no_vocals_file = os.path.join(song_path, "no_vocals.mp3")

    if os.path.exists(vocals_file) and os.path.exists(no_vocals_file):
        vocals_feat = extract_features(vocals_file)
        no_vocals_feat = extract_features(no_vocals_file)

        # Assuming the genre is not available from the directory structure,
        # we'll use a placeholder for now.
        genre = "unknown"

        data.append({
            "vocals": vocals_feat.tolist(),
            "no_vocals": no_vocals_feat.tolist(),
            "class": genre
        })
    else:
        print(f"Skipping {song_name}: vocal or no_vocals file not found.")


# Create DataFrame
df = pd.DataFrame(data)

# Prepare X, y for training
X = [df.iloc[i]["vocals"] + df.iloc[i]["no_vocals"] for i in range(len(df))]
X = np.array(X)

# If you have true genre labels, replace the dummy labels below.
# For now, we'll use dummy labels for demonstration.
y_dummy = np.random.choice(["sad", "romantic", "rap", "melody"], size=len(X))


# Encode labels
encoder = LabelEncoder()
y_enc = encoder.fit_transform(y_dummy)

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y_enc, test_size=0.2, random_state=42)

# Train model
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict genres
y_pred = clf.predict(X)
df["class"] = encoder.inverse_transform(y_pred)

# Save CSV
df.to_csv(csv_output, index=False)

print(f"✅ Dataset with predicted genres saved to {csv_output}")
print(df.head())

In [ ]:
import pandas as pd

In [ ]:
pd = pd.read_csv("/content/dataset.csv")

In [ ]:
pd